In [1]:
from transformers import (
    AutoTokenizer,
    BitsAndBytesConfig,
)
from peft import AutoPeftModelForSequenceClassification
from datasets import load_dataset
import torch
from tqdm.notebook import tqdm

/fs01/home/dadsetan/condaenvs/lora-finetuning-mia/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/fs01/home/dadsetan/condaenvs/lora-finetuning-mia/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
id2label = {0: "negative", 1: "neutral", 2: "positive"}
label2id = {"negative": 0, "neutral": 1, "positive": 2}
device = "cuda"
dp_model = AutoPeftModelForSequenceClassification.from_pretrained(
    "/projects/privacyllm/tweet_shield_dp_trained_model/checkpoint-12267",
    num_labels=3,
    quantization_config=bnb_config
)
dp_model = dp_model.to(device)
dp_model.eval()
dp_model.config.pad_token_id = dp_model.config.eos_token_id


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at /model-weights/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
normal_model = AutoPeftModelForSequenceClassification.from_pretrained(
    "/projects/privacyllm/tweet_shield_normal_trained_model/checkpoint-13640",
    num_labels=3,
    quantization_config=bnb_config
)
normal_model = normal_model.to(device)
normal_model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at /model-weights/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): LlamaForSequenceClassification(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
          

In [4]:
tokenizer = AutoTokenizer.from_pretrained("/model-weights/Llama-2-7b-hf", return_tensors="pt", 
                                          model_max_length=150)

tokenizer.pad_token = tokenizer.eos_token

In [27]:
dataset["train"].shuffle()[:5]["text"]

['united is changing its  mileage plus program  winners and losers  airline  travel  loyalty  rewards',
 'southwest air from lga to san diego',
 '"i\'m so glad there are so many out gay men on a&amp;e\'s &quot;obsessed.&quot; i relate to russ. this is very difficult to watch. "',
 "southwest air flying flight 3130 tonight at 7 20 from pbi  i have boarding position c 42. is it overbooked? really don't want to be bumped!",
 'southwest air i will direct message you now']

In [16]:
orig_dataset = load_dataset("parquet", data_files="data_balanced.parquet")

dataset = orig_dataset.rename_column("Lable", "label").rename_column("Text", "text")
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)

# def preprocess_fn(examples):
#     return tokenizer(examples["text"], return_tensors='pt')

def change_labels(example):
  label = example["label"]
  n_label = 0 if label == "negative" else 1 if label == "neutral" else 2
  example["label"] = n_label
  return example

# toked = toked.map(change_labels)

# toked = dataset.map(preprocess_fn)

In [ ]:
def print_results(data, dp_label, normal_label):
    print(data["text"])
    print(f"correct label: {id2label[data['label']]}")
    print(f"dp label:      {id2label[dp_label]}")
    print(f"normal label:  {id2label[normal_label]}")
    print("="*60)

both_wrong = []
dp_wrong = []
normal_wrong = []
all_results = []

for data in tqdm(toked["test"]):
    input_ids = torch.tensor(data["input_ids"]).to("cuda")
    
    dp_logits = dp_model(input_ids=input_ids)["logits"]
    dp_label = torch.argmax(dp_logits,dim=-1)[0].item()
    
    dp_correct = (dp_label == data["label"])

    normal_logits = normal_model(input_ids=input_ids)["logits"]
    normal_label = torch.argmax(normal_logits,dim=-1)[0].item()

    normal_correct = (normal_label == data["label"])

    if normal_correct and dp_correct:
        continue

    if normal_correct:
        dp_wrong.append((data, dp_label, normal_label))
    elif dp_correct:
        normal_wrong.append((data, dp_label, normal_label))
    else:
        both_wrong.append((data, dp_label, normal_label))

    all_results.append((data, dp_label, normal_label))

In [ ]:
len(both_wrong)

In [ ]:
len(normal_wrong)

In [ ]:
len(dp_wrong)

In [ ]:
for t in dp_wrong:
    if t[0]["label"] == 0:
        print_results(*t)
        

In [ ]:
from random import shuffle